# Entraînement de Yolo V7 sur des données personnelles

Vous avez très envie d'utiliser Yolo V7 sur votre problématique de reconnaissance d'objet ou de segmentation mais vous n'avez pas de gros ordinateur avec une groooosse carte graphique pour faire de grooooooooooos calculs d'optimisation de poids. Pas de panique, il y a des solutions et on vous en propose une ici.

## Utilisation du GPU

Le petite magicien qui rend l'entraînement des modèles de deep learning très profonds, c'est lui : le GPU. *Cool mais déjà c'est quoi ? Et puis comment je fais si j'en ai pas ?*

### Le GPU

Une unité de traitement graphique ou GPU (*Graphics Processing Unit*) est une puce informatique pour traiter les tâches de rendu graphique. Les GPU sont conçus pour effectuer de nombreux calculs simultanément, ce qui les rend particulièrement efficaces pour le rendu graphiques 2D/3D ou le traitement de vidéos mais pas seulement. Le GPU fonctionne conjointement avec le CPU et permet, en fonctionnant spécialement pour le rendu images, de libérer de la puissance de traitement pour le CPU qui peut se consacrer aux autres tâches sans limiter les performances de la carte graphique.

Le GPU est généralement disposé sur la carte graphique (d'où la confusion parfois entre les 2) mais pas nécessairement. En effet la puce GPU peut être intégrée à un CPU sur le même circuit, sur une carte graphique ou dans la carte mère d'un ordinateur ou d'un serveur. 

### GPU, CPU, JSUIPERDU...

Un GPU est plus efficace qu'un CPU pour le rendu d'images grâce à son architecture de traitement parallèle lui permettant d'effectuer de nombreux calculs simultanément. Un seul CPU ne dispose pas de cette fonctionnalité (bien que ce soit possible avec des processeurs multicœurs). En revanche un CPU a une fréquence plus élevée et peut effectuer un calcul plus rapidement qu'un GPU.

Pour résumer, le GPU est conçu pour le parallélisme des données et pour appliquer la même opération à plusieurs éléments de données (SIMD pour *Single Instruction to Multiple Data*) tandis qu'un CPU est conçu pour le parallélisme des tâches et l'exécution de différentes opérations non liées.

Si votre ordinateur est forcément équipé d'un CPU, il n'a pas nécessairement de GPU puisque le chipset de la carte mère peut gérer le rendu graphique (beaucoup moins bien qu'une carte graphique certes).

### GPU et *Deep Learning*

*"Bon ok, je suis toujours pas spécialiste, mais je vois le principe. Par contre, on était pas sur YoloV7 nous ?"*  

YoloV7 est un réseau de neurones à convolution qui a quasi 37 millions de paramètres...lorsque vous souhaitez utiliser le modèle déjà entraîné sur le jeu de données [COCO](https://cocodataset.org/#home), pas de problèmes de hardware, ça fonctionnera sans ressources supplémentaires. Si par contre vous voulez ré-entraîner le modèle sur vos données, là ça se complique et si vous n'avez pas de GPU, vous avez intérêt à avoir énormément de temps devant vous, et d'ailleurs ça ne suffira même pas, car même en ayant le temps, vous risquez de manquer de mémoire.

Sans entrer dans les détails de l'optimisation de réseau de neurones, les opérations à effectuer sont des calculs relativement simple mais très répétitifs et surtout, extrêmement nombreux. La parallélisation de ces opérations sur un GPU est donc idéale et nécessaire pour optimiser le réseau dans un temps viable.

### Les sevices de GPU cloud

*"On y voit un peu plus clair...mais comment on fait quand on en a pas ?"*

Puisqu'on a pas les capacités localement, on va le faire à distance grâce aux services Cloud qui proposent du GPU. Ils sont nombreux (Linode, Paperspace, Google Cloud GPUs, Elastic GPU Service, Azure N series, IBM Cloud, AWS and NVIDIA, OVHcloud, Lambda GPU, Genesis Cloud, etc...) mais on va s'en tenir a un : [Paperspace](https://www.paperspace.com/). L'intérêt est d'utiliser [Paperspace Gradient](https://www.paperspace.com/gradient) qui facilite l'utilisation et l'entraînement de modèles à l'aide de notebooks. Si on le souhaite, on peut louer directement du GPU avec Paperspace CORE.

## Cas pratique : la reconnaissance de déchets

L'objectif est d'entraîner le modèle Yolo V7 à identifier et reconnaître des déchets. On utilisera pour cela le dataset [TACO](http://tacodataset.org/). On passera assez vite sur la partie de récupération, de restructuration des dossiers et le passage des annotations du format json vers le format yolo mais tout le code est évidemment disponible.






In [2]:
import json
import os
import random
from tqdm import tqdm
import shutil
import datetime
import re

## Yolo V7

On clone directement le repo de [yolov7](https://github.com/WongKinYiu/yolov7.git) pour pouvoir réentrainer le modèle sur nos données.

In [3]:
!git clone https://github.com/WongKinYiu/yolov7.git

Clonage dans 'yolov7'...
remote: Enumerating objects: 1127, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1127 (delta 12), reused 14 (delta 4), pack-reused 1098
Réception d'objets: 100% (1127/1127), 69.96 Mio | 16.98 Mio/s, fait.
Résolution des deltas: 100% (522/522), fait.


Puis on installe le requirements. Selon la machine GPU que vous sélectionnez sur paperspace Gradient, vous pouvez avoir besoin (ou pas) de downgrader les versions Torch et Torchvision. Ici c'est le cas avec une VM A4000.

In [3]:
!pip install -r ./yolov7/requirements.txt
!pip install setuptools==59.5.0
!pip install torchvision==0.11.3+cu111 -f https://download.pytorch.org/whl/cu111/torch_stable.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.4/776.4 MB 3.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 73.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 1.12.0+cu116
    Uninstalling torch-1.12.0+cu116:
      Successfully uninstalled torch-1.12.0+cu116
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.0+cu116
    Uninstalling torchvision-0.13.0+cu116:
      Successfully uninstalled torchvision-0.13.0+cu116
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 0.12.0+cu116 requires torch==1.12.0, but you have torch 1.12.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 70.3 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 63.1.0
    Uninstalling 

## Train test split

On fait le découpage à la main à partir des noms d'images et on met directement les datasets `train`, `val`, `test` ainsi que le fichier .yaml associé dans le repo yolov7 pour l'entraînement.

In [4]:
# Split dataset

# read json file
with open('./TACO/data/images/annotations_wo_subdir.json', 'r+') as file:
    json_file = json.load(file)
    
# create directories (with replacement if exists)
for dirname in ['train', 'val', 'test']:
    dirpath = f"./yolov7/data/TACOpoly/{dirname}"
    if os.path.exists(dirpath):
        shutil.rmtree(dirpath)
    os.makedirs(dirpath + '/images')
    os.makedirs(dirpath + '/labels')
    
# create yaml file (with replacement if exists)
cats = [cat['name'] for cat in json_file['categories']]

with open('./yolov7/data/TACOpoly.yaml', 'w') as f:
    f.write(
f"""train: ./data/TACOpoly/train/images
val: ./data/TACOpoly/val/images
test: ./data/TACOpoly/test/images

nc: {len(cats)}
names: {cats}""")
    
    
# read json annotations file
with open('./TACO/data/images/annotations_wo_subdir.json', 'r+') as file:
    json_file = json.load(file)

# get images names and shuffle
img_names = [img['file_name'].split('.')[0] for img in json_file['images']]
random.shuffle(img_names)

# create a splitting dictionnary
split = {
    'train' : img_names[:1200],
    'val' : img_names[1200:1400],
    'test' : img_names[1400:]
}

# copy each image and its label in the right directory
for setname, sample in split.items():
    print(f"Copying images to {setname.upper()} directory")
    for imgname in tqdm(sample):
        shutil.copy(f"./TACO/data/images/{imgname}.jpg", f"./yolov7/data/TACOpoly/{setname}/images/{imgname}.jpg")
        shutil.copy(f"./TACO/data/labels_poly/{imgname}.txt", f"./yolov7/data/TACOpoly/{setname}/labels/{imgname}.txt")

Copying images to TRAIN directory


100%|██████████| 1200/1200 [01:50<00:00, 10.84it/s]


Copying images to VAL directory


100%|██████████| 200/200 [00:17<00:00, 11.56it/s]


Copying images to TEST directory


100%|██████████| 100/100 [00:08<00:00, 11.31it/s]


## Entraînement (transfer learning)

On doit dans un premier temps téléchargement les poids initiaux du modèle pré-entraîné puis lancer l'entraînement sur nos données.

In [4]:
%cd /notebooks/yolov7

/notebooks/yolov7


In [5]:
if os.path.exists('yolov7_training.pt'):
    print("Déjà téléchargé")
else:
    !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

Déjà téléchargé


In [6]:
def get_last_weights(modelname):
    """
    This function retrieves the best weights from the last training in order to
    restart new traing from those weights.
    
    Parameters
    ----------
    modelname : str
        Name of the model (such as --name argument from Yolov7 train.py script).
        This is the name looked for in the yolov7/runs/train directory.
    nb_epochs : int
        Number of epochs done per each training.

    Returns
    -------
    str :
        Path to weights used to initiate new training.
    int :
        Number of epochs already trained.  
    """
        
    # keep only directories containg modelname in their name
    train_dirs = []
    for dirname in os.listdir('/notebooks/yolov7/runs/train/'):
        if modelname in dirname:
            train_dirs.append(dirname)
    train_dirs.sort()
    
    # returns yolov7_training weights and 0 epochs if never trained
    if len(train_dirs) == 0:
        return 'yolov7_training.pt', 0

    # else retrieve the last weights and compute number of epochs
    # this assumes that the number of epochs is the same over each training
    nmax = 0
    for dirname in train_dirs:
        if dirname.split(modelname)[-1] == '' :
            dirmax = dirname
        else:
            n = int(dirname.split(modelname)[-1])
            if n > nmax :
                nmax = n
                dirmax = dirname

    return f"runs/train/{dirmax}/weights/best.pt"

In [7]:
epochs_per_training = 150
init_weights = get_last_weights('TACOpoly')
start = datetime.datetime.now()

print(f"ENTRAÎNEMENT DÉBUTÉ À {start.strftime('%H:%M')} AVEC LES POIDS INITIAUX {init_weights}")
print(f"_________________________________________________________________")


!python train.py --workers 8 --device 0 --batch-size 16 --data data/TACOpoly.yaml --img 640 640 \
    --cfg cfg/training/yolov7.yaml --weights {init_weights} --name TACOpoly \
    --hyp data/hyp.scratch.custom.yaml --epochs {epochs_per_training}

print(f"_________________________________________________________________")
print(f"DURÉE DE L'ENTRAÎNEMENT : {datetime.datetime.now() - start}")

ENTRAÎNEMENT DÉBUTÉ À 10:24 AVEC LES POIDS INITIAUX runs/train/TACOpoly7/weights/best.pt
_________________________________________________________________
YOLOR 🚀 v0.1-115-g072f76c torch 1.10.2+cu111 CUDA:0 (NVIDIA RTX A4000, 16117.3125MB)

Namespace(weights='runs/train/TACOpoly7/weights/best.pt', cfg='cfg/training/yolov7.yaml', data='data/TACOpoly.yaml', hyp='data/hyp.scratch.custom.yaml', epochs=150, batch_size=16, img_size=[640, 640], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='0', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='TACOpoly', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/TACOpoly8', total_batch_size=16)
tens

Si l'entraînement s'est bien terminé sans erreur, on peut supprimer le contenu du dossier de l'entraînement précédent pour éviter de surcharger le stockage du compte Paperspace Gradient.

Il faut en revanche conserver le dossier, même vide car sinon les nouveaux entraînement seront stockés dans ces dossiers là (c'est dû à la méthode d'indentation des dossier dans le code source de yolov7).

C'est d'ailleurs tout l'intérêt de cette approche puisqu'il existe sinon la possibilité de reprendre l'entraînement où il s'était arrêté (`!python train.py --resume`).

In [8]:
last_weights = get_last_weights('TACOpoly')

if (init_weights != 'yolov7_training.pt') & os.path.exists(last_weights):
    dir_to_empty = os.path.dirname(os.path.dirname(init_weights))
    shutil.rmtree(dir_to_empty)
    os.makedirs(dir_to_empty)